# Paperspace用 ComfyUI 起動ノートブック（最終・全部入り版）

**特徴：**
- 必要なツールやライブラリを全て自動でインストール
- 必須拡張機能「ComfyUI-Manager」も自動でインストール
- 基本モデルを自動でダウンロードするので、すぐに画像生成を試せます
- **あなたのGoogleドライブから、好きなモデルを追加することもできます！**

**使い方：**
1. 上のメニューから `実行` → `すべてのセルを実行` をクリックしてください。
2. 初回は15分以上かかります。じっくりお待ちください。
3. 最後のセルでComfyUIが起動します。

In [ ]:
# Step-1： 環境の準備（ビルドツール等のインストール）
!apt-get update -y
!apt-get install -y git wget build-essential cmake libgl1 libglib2.0-0

In [ ]:
# Step-2： ComfyUI本体と拡張機能（Manager）のインストール
import os
COMFYUI_DIR = '/notebooks/ComfyUI'
if not os.path.exists(COMFYUI_DIR):
    !git clone --depth 1 https://github.com/comfyanonymous/ComfyUI.git {COMFYUI_DIR}
else:
    !cd {COMFYUI_DIR} && git pull

MANAGER_DIR = os.path.join(COMFYUI_DIR, 'custom_nodes', 'ComfyUI-Manager')
if not os.path.exists(MANAGER_DIR):
    !git clone https://github.com/ltdrdata/ComfyUI-Manager.git {MANAGER_DIR}

In [ ]:
# Step-3： PyTorchと依存ライブラリのインストール（安定版）
import subprocess
cuda_available = subprocess.run(['nvidia-smi', '-L'], capture_output=True).stdout.decode('utf-8')
if cuda_available:
    !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
else:
    !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install --upgrade pip wheel setuptools gdown
!pip install -r {COMFYUI_DIR}/requirements.txt --prefer-binary
if cuda_available:
    !pip install xformers==0.0.23.post1 --extra-index-url https://download.pytorch.org/whl/cu118

In [ ]:
# Step-4： 基本モデルの自動ダウンロード（とりあえず動かしたい人向け）
TARGET_DIR=f"{COMFYUI_DIR}/models/checkpoints"
os.makedirs(TARGET_DIR, exist_ok=True)
MODEL_URL="https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors"
!wget -nc -O {TARGET_DIR}/sd15_v1.5.safetensors {MODEL_URL}

### 【オプション】Step-5： あなたのモデルをGoogleドライブから追加
自分の好きなモデルを使いたい場合は、下のセルを実行してください。
1. Googleドライブで使いたいモデルファイルを右クリック → `共有` → `リンクをコピー`
2. 下の `gdrive_url = "..."` の中に、コピーしたリンクを貼り付けます。
3. ファイル名も好きな名前に変更できます（拡張子は `.safetensors` や `.ckpt` にしてください）。
4. 準備ができたら、下のセルだけを実行（▷ボタンを押す）してください。

In [ ]:
# ▼▼▼ ここを編集してください ▼▼▼
gdrive_url = "ここにGoogleドライブの共有リンクを貼り付けてください"
output_filename = "my_model.safetensors"
# ▲▲▲ ここまで ▲▲▲

if "drive.google.com" in gdrive_url:
    print(f"Googleドライブから {output_filename} をダウンロードしています...")
    !gdown {gdrive_url} -O {TARGET_DIR}/{output_filename}
    print("✅ ダウンロードが完了しました！")
else:
    print("⚠️ 正しいGoogleドライブの共有リンクを貼り付けてください。")

In [ ]:
# Step-6： ComfyUIの起動
!cd {COMFYUI_DIR} && python main.py --listen --port 8889 --disable-auto-launch